In [28]:
import pandas as pd
import re
from datetime import datetime

In [40]:
df = pd.read_csv("../data/raw/Motor_Vehicle_Collisions_-_Crashes.csv")

# drop useless columns
df = df.drop(columns=["ZIP CODE", "LATITUDE", "LONGITUDE", "ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME"])

# convert "CRASH DATE" from object to datetime
df[["CRASH DATE"]] = pd.to_datetime(df[["CRASH DATE"]].stack()).unstack()

/tmp/ipykernel_22108/3839166150.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/Motor_Vehicle_Collisions_-_Crashes.csv")


In [71]:
# so that filtering out non-2019 crashes is a lot easier
df = df[df["CRASH DATE"] > datetime(2018, 12, 31)]
df = df[df["CRASH DATE"] < datetime(2020, 1, 1)]

# the time of the crash needs to be converted to a consistent format (e.g. 2:10 should be 02:10)
# the "CRASH TIME" column is located at index 1
for i in range(df.shape[0]):
    time = df.iat[i, 1]
    if len(time) == 4:
        time = f"0{time}"
        df.iat[i, 1] = time

df.head()


,CRASH DATE,CRASH TIME,BOROUGH,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
20,2019-05-21,22:50,BROOKLYN,"(40.69754, -73.98312)",0.0,0.0,0,0,0,0,...,Unspecified,NaN,NaN,NaN,4136992,�MBU,Taxi,NaN,NaN,NaN
1059,2019-04-17,00:49,NaN,"(40.651974, -73.86542)",3.0,0.0,0,0,0,0,...,Unspecified,NaN,NaN,NaN,4408571,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
20389,2019-07-22,08:20,BROOKLYN,"(40.615433, -73.91388)",0.0,0.0,0,0,0,0,...,Failure to Yield Right-of-Way,NaN,NaN,NaN,4548461,Sedan,Sedan,NaN,NaN,NaN
33658,2019-10-19,17:20,NaN,NaN,0.0,0.0,0,0,0,0,...,Unspecified,NaN,NaN,NaN,4226690,Sedan,Sedan,NaN,NaN,NaN
177966,2019-12-18,19:45,NaN,NaN,1.0,0.0,1,0,0,0,...,NaN,NaN,NaN,NaN,4260765,Sedan,NaN,NaN,NaN,NaN


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211486 entries, 20 to 513118
Data columns (total 23 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   CRASH DATE                     211486 non-null  datetime64[ns]
 1   CRASH TIME                     211486 non-null  object        
 2   BOROUGH                        137024 non-null  object        
 3   LOCATION                       194114 non-null  object        
 4   NUMBER OF PERSONS INJURED      211486 non-null  float64       
 5   NUMBER OF PERSONS KILLED       211486 non-null  float64       
 6   NUMBER OF PEDESTRIANS INJURED  211486 non-null  int64         
 7   NUMBER OF PEDESTRIANS KILLED   211486 non-null  int64         
 8   NUMBER OF CYCLIST INJURED      211486 non-null  int64         
 9   NUMBER OF CYCLIST KILLED       211486 non-null  int64         
 10  NUMBER OF MOTORIST INJURED     211486 non-null  int64         
 11 

In [95]:
# I've taken a look at all the distinct values for vehicle type codes, and vehicles involving taxis are labelled "Taxi"
#df["VEHICLE TYPE CODE 1"].unique()

# find out how many crashes involve taxis
taxi_crashes = df.loc[(df["VEHICLE TYPE CODE 1"] == "Taxi") | (df["VEHICLE TYPE CODE 2"] == "Taxi")| (df["VEHICLE TYPE CODE 3"] == "Taxi") \
    | (df["VEHICLE TYPE CODE 4"] == "Taxi") | (df["VEHICLE TYPE CODE 5"] == "Taxi")].shape[0]
total_crashes = df.shape[0]

taxi_crashes_perc = f"{round(taxi_crashes / total_crashes * 100, 2)}%"
taxi_crashes_perc

# and clean the columns further by making it binary
for j in range(18, 23):
    for i in range(df.shape[0]):
        code = df.iat[i, j]
        if code != "Taxi":
            df.iat[i, j] = "Not Taxi"

df.sample(5)

KeyboardInterrupt: 